## Import Librarires

In [1]:
import pandas as pd
import altair as alt
import osmnx as ox
import folium
from folium import plugins
from folium import plugins
import geopandas as gpd
import pandas as pd
import ipywidgets as widgets
from IPython.display import display



### Data Extraction

In [2]:
# Fetch the data for the Seattle University
### R5282350 is the area code for the seattle university 
seattle = ox.geocode_to_gdf('R5282350', by_osmid = True)




# Parks
### Extract the information about the Parks in the Seattle University

parks = ox.features_from_polygon(seattle["geometry"].iloc[0], tags={"leisure": "park"})
parks = gpd.GeoDataFrame.from_features(parks)
parks = parks[parks.geom_type == 'Polygon']
parks = gpd.GeoDataFrame.from_features(parks)
### Renamed the park by removing the street information as we are going to use this later to display
### the name on the graph
parks.loc[parks['name'] == '12th Ave Square Park', 'name'] = 'Square Park'


### Buildings
### Extract the information about the Buildings in the Seattle University

buildings = ox.features_from_polygon(seattle["geometry"].iloc[0], tags={"building": True})
buildings = gpd.GeoDataFrame.from_features(buildings)
buildings = buildings[buildings.geom_type == 'Polygon']
buildings = gpd.GeoDataFrame.from_features(buildings)
# Removing the names of the buildings which doesn't have any name
# I was also getting the roofs of the building, which is too much information on the graph
buildings = buildings[buildings['name'].notna()]



### Cafes
### Extract the information about the Cafes in the Seattle University
cafes = ox.features_from_polygon(seattle["geometry"].iloc[0], tags={"amenity":['cafe']})
cafes['geometry'] = cafes['geometry'].representative_point()
cafes = gpd.GeoDataFrame.from_features(cafes)
cafes['lon'] = cafes['geometry'].x
cafes['lat'] = cafes['geometry'].y



### Cycle Parking
### Extract the information about the Cycle Parking in the Seattle University

cycle_parking = ox.features_from_polygon(seattle["geometry"].iloc[0], tags={"amenity": "bicycle_parking"})
cycle_parking = gpd.GeoDataFrame.from_features(cycle_parking)
cycle_parkingpts = cycle_parking[cycle_parking.geom_type == 'Point']
cycle_parkingpts['lon']=cycle_parkingpts['geometry'].x
cycle_parkingpts['lat']=cycle_parkingpts['geometry'].y
cycle_parkingpts = cycle_parkingpts[cycle_parkingpts['operator'].notna()]

### Map in Folium

In [3]:
# Intialize the map and set the location of the seattle University
m = folium.Map(location=[47.6095, -122.3171], 
               tiles='cartodbpositron',
               zoom_start=17, control_scale=True,
               width=900,
               height=1100)

# Cluster to show the datapoints
marker_cluster = plugins.MarkerCluster().add_to(m)


## Plotting Cycle parking points
for idx, row in cycle_parkingpts.iterrows():
    # Get lat and lon of points
    lon = row['lon']
    lat = row['lat']
    icon = folium.Icon(icon = 'bicycle', color = 'blue', prefix = 'fa')
    # Add marker to the map NOTE USE OF MARKER CLUSTER HERE
    folium.Marker(
        location=[lat, lon],        
        icon=icon,
    ).add_to(marker_cluster)


    
## Boundary of the seattle university 
for idx, row in seattle.iterrows():

    # get coordinates of the polygon geometry into a plain list
    x, y = row['geometry'].exterior.coords.xy
    locs = tuple(zip(y,x))
    
    name = row['name']
    
    folium.Polygon(
        locations=locs,
        color="red",
        weight=1,
        #fill_color="#00000000",
        fill=False,
        tooltip=name
    ).add_to(m)
    
    
## Add information about the cafes Icon
for idx, row in cafes.iterrows():
    # Get lat and lon of points
    lon = row['lon']
    lat = row['lat']
    icon = folium.Icon(icon = 'mug-saucer', color = 'black', prefix = 'fa')
    # Add marker to the map NOTE USE OF MARKER CLUSTER HERE
    folium.Marker(
        location=[lat, lon],
        tooltip = row['name'],
        icon=icon,
    ).add_to(m)

## Code to add the name of cafes on the map
for idx, row in cafes.iterrows():
    lon = row['lon']
    lat = row['lat']
    name = row['name']

    # Add a text label with the park's name at the center of the park
    folium.Marker(
        location=[lat, lon],
        icon=folium.DivIcon(html=f'<div>{name.replace("<BR>", " ")}</div>'),
    ).add_to(m)
    

## Code to add the info about the parks in the college
for idx, row in parks.iterrows():
    # get coordinates of the polygon geometry into a plain list
    x, y = row['geometry'].exterior.coords.xy
    locs = tuple(zip(y,x))
    
    name = row['name']
    
    folium.Polygon(
        locations=locs,
        color="darkgreen",
        weight=1,
        fill_color="lightgreen",
        fill_opacity=0.5,
        fill=True,
        tooltip=name
    ).add_to(m)
    

## Code to add the name of the parks on the map    
for idx, row in parks.iterrows():
    # Get coordinates of the polygon geometry
    x, y = row['geometry'].centroid.coords.xy
    lat, lon = y[0], x[0]  # Calculate the centroid coordinates

    name = row['name'].replace("\n", " ")
    name = row['name'].replace(" ", "&nbsp;")

    # Add a text label with the park's name at the center of the park
    folium.Marker(
        location=[lat, lon],
        icon=folium.DivIcon(html=f'<div style="white-space: nowrap;">{name}</div>'),
    ).add_to(m)
    
    
    
## Add building data on the graph
for idx, row in buildings.iterrows():

    # get coordinates of the polygon geometry into a plain list
    x, y = row['geometry'].exterior.coords.xy
    locs = tuple(zip(y,x))
    
    name = row['name']
    if name == 'Chapel of Saint Ignatius':
        #folium.Marker(locations=locs)
        icon = folium.Icon(icon = 'church', color = 'black', prefix = 'fa')
        # Add marker to the map NOTE USE OF MARKER CLUSTER HERE
        folium.Marker(
            location=[47.6114965,-122.31800 ],
            icon=icon,
        ).add_to(m)
    
    folium.Polygon(
        locations=locs,
        color="darkgreen",
        weight=1,
        fill_color="#aa0000",
        fill_opacity=0.5,
        fill=True,
        tooltip=name
    ).add_to(m)
    
    




## Code to add the Latitude Longitude Information on the map
m.add_child(folium.LatLngPopup())


m

In [4]:
 m.save('seattleU.html')